In [1]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import imageio

/home/huqiang/anaconda3/envs/SLTnet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# GIF generate

In [2]:
def plot_bboxes(image, bboxes, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(
        0.002 * (image.shape[0] + image.shape[1]) / 2) + 1  # line/font thickness
    for (x1, y1, x2, y2, cls_id, pos_id) in bboxes:
        if cls_id in ['person']:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
        c1, c2 = (x1, y1), (x2, y2)
        cv2.rectangle(image, c1, c2, (255,255,255), thickness=1, lineType=cv2.LINE_AA)
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(cls_id, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(image, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(image, '{} ID-{}'.format(cls_id, pos_id), (c1[0], c1[1] - 2), 0, tl / 3,
                    [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
        
        mask = np.zeros(image.shape)
        mask[y1:y2,x1:x2,0] = 238
        mask[y1:y2,x1:x2,1] = 130
        mask[y1:y2,x1:x2,2] = 238
        
        # contours,_ = cv2.findContours(np.uint8(mask),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        # cv2.drawContours(image,contours,-1,(238,130,238),1)
        image = cv2.add(image,np.uint8(mask))

In [3]:
def plot(img, mask=None):
    if mask is None:
        return cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    else:
        mask_edge, _= cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        alpha = 0.6
        purple_mask = np.zeros_like(img)# purple mask[mask==255]=(226.43,138)
        purple_mask[mask==255]=(225,61,72)# overlay = img.copy()
        output =cv2.addWeighted(img, alpha, purple_mask,1- alpha, 0)
        cv2.drawContours(output,mask_edge,-1,(255,255,255),thickness=2)#绘制预测mask的边界
        # plt.imshow(output)
        
        return cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
    # cv2.imwrite('merge.png', output)

In [10]:
img_root = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestHardDataset/Unseen/Frame/'
models = ['SALI','MAST','PNS+','PNS-Net','23D']
pred_root = ['/data/yizhenyu/project/video-polyp-seg/SLT-Net_align_Memory/res/cas_long2short/Net_epoch_4_best/SUN-SEG/TestHardDataset/Unseen/'
,           '/data/yizhenyu/datasets/pvt_MAST/TestHardDataset/Unseen/'
,           '/data/yizhenyu/datasets/Benchmark/2022-MIR-PNS+/TestHardDataset/Unseen/'
,           '/data/yizhenyu/datasets/Benchmark/2021-MICCAI-PNSNet/TestHardDataset/Unseen/'
,           '/data/yizhenyu/datasets/Benchmark/2020-MICCAI-23DCNN/TestHardDataset/Unseen/'
]

image_sequence = []
gt_sequence = []
pred_sequence = [ [] for i in range(len(pred_root))]

############ 

# select, ranges = 'case29_1/case_M_20181018093806_0U62363101890605_1_007_002-1_a10_ayy_image', range(34,95)
# select, ranges = 'case32_4/case_M_20181019101517_0U62367101935917_1_003_001-1_a15_ayy_image', range(50,111)
select, select_ranges = 'case68_4/case_M_20181214095357_0U62368121473456_1_006_003-1_a18_ayy_image', range(33,94)

############


for j in select_ranges:
    sample = select + f'{"%04d" % j}' + '.jpg'
    
    img_path = img_root + sample
    img = cv2.imread(img_path)

    gt_path = img_path.replace('Frame','GT').replace('.jpg','.png')
    gt = cv2.imread(gt_path,cv2.IMREAD_GRAYSCALE)

    for idx in range(len(models)): 
        mask_path = pred_root[idx] + sample.replace('.jpg','.png')
        pred = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)

        if not gt.shape == pred.shape:
            pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]))
        pred = np.uint8(pred > 128)*255
        pred_sequence[idx].append(plot(img,pred))


    image_sequence.append(plot(img))
    gt_sequence.append(plot(img,gt))
    

In [12]:
imageio.mimsave(select.split('/')[0]+'/'+select.split('/')[1].split('_')[-3]+'_'+'23D'+'_pred.gif', pred_sequence[idx], duration=0.1, loop=0)

In [9]:
os.makedirs(select.split('/')[0],exist_ok=True)
imageio.mimsave(select.split('/')[0]+'/'+select.split('/')[1].split('_')[-3]+ '_img.gif', image_sequence, duration=0.1, loop=0)
imageio.mimsave(select.split('/')[0]+'/'+select.split('/')[1].split('_')[-3]+'_gt.gif',gt_sequence, duration=0.1, loop=0)

for idx, model in enumerate(models):
    imageio.mimsave(select.split('/')[0]+'/'+select.split('/')[1].split('_')[-3]+'_'+model+'_pred.gif', pred_sequence[idx], duration=0.1, loop=0)

FileNotFoundError: The directory '/data/yizhenyu/project/tools/keshihua/case68_4/a18_2' does not exist

In [20]:
select = 'case68_4/case_M_20181214095357_0U62368121473456_1_006_003-1_a18_ayy_image'
i=1
select + f'{"%04d" % i}' + '.png'

'case68_4/case_M_20181214095357_0U62368121473456_1_006_003-1_a18_ayy_image0001.png'

In [18]:
from moviepy.editor import VideoFileClip
import moviepy.editor as mp

# 读取 MP4 视频文件
clip = VideoFileClip("demo.mp4")

# 将视频转换为 GIF，并设置循环次数为 0，表示无限循环播放
clip.write_gif("demo.gif", fps=30, loop=0)

# # 在此处将 gif 转换为 mp4 文件
# clip.write_videofile("output.mp4", fps=10, codec='libx264', audio_codec='aac')

MoviePy - Building file demo.gif with imageio.


In [21]:
reader = imageio.get_reader('demo.mp4')

In [22]:
reader

In [25]:
fps = reader.get_meta_data()['fps']
with imageio.get_writer('output2.gif', duration=0.1,loop=0) as writer:
    for frame in reader:
        writer.append_data(frame)

# change weights-key

In [1]:
import torch

model_dir = '../../video-polyp-seg/LSI-Net/snapshot/model_changed.pth'
dict = torch.load(model_dir,map_location=lambda storage, loc: storage.cuda(1))

/home/huqiang/anaconda3/envs/SLTnet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for key in list(dict.keys()):
    if key.startswith('ls_fusion_conv'):
        del dict[key]

In [7]:
old_key = ["nlnet.nl_layer1.nonlocals.0.g.0.weight",  "nlnet.nl_layer1.nonlocals.0.g.0.bias", 
           "nlnet.nl_layer1.nonlocals.0.W.weight", "nlnet.nl_layer1.nonlocals.0.W.bias", 
           "nlnet.nl_layer1.out_conv.weight", "nlnet.nl_layer1.out_conv.bias", 

           "nlnet.nl_layer2.nonlocals.0.g.0.weight", "nlnet.nl_layer2.nonlocals.0.g.0.bias", 
            "nlnet.nl_layer2.nonlocals.0.W.weight", "nlnet.nl_layer2.nonlocals.0.W.bias", 
            "nlnet.nl_layer2.out_conv.weight", "nlnet.nl_layer2.out_conv.bias", 

            "nlnet.nl_layer3.nonlocals.0.g.0.weight", "nlnet.nl_layer3.nonlocals.0.g.0.bias", 
            "nlnet.nl_layer3.nonlocals.0.W.weight", "nlnet.nl_layer3.nonlocals.0.W.bias", 
            "nlnet.nl_layer3.out_conv.weight", "nlnet.nl_layer3.out_conv.bias"]

new_key = ["nlnet.nl_layer1.g.0.weight", "nlnet.nl_layer1.g.0.bias", 
           "nlnet.nl_layer1.W.0.weight", "nlnet.nl_layer1.W.0.bias", 
           "nlnet.nl_layer1.residual_conv.weight", "nlnet.nl_layer1.residual_conv.bias", 
        
           "nlnet.nl_layer2.g.0.weight", "nlnet.nl_layer2.g.0.bias",          
           "nlnet.nl_layer2.W.0.weight", "nlnet.nl_layer2.W.0.bias", 
           "nlnet.nl_layer2.residual_conv.weight", "nlnet.nl_layer2.residual_conv.bias", 

            "nlnet.nl_layer3.g.0.weight", "nlnet.nl_layer3.g.0.bias", 
            "nlnet.nl_layer3.W.0.weight", "nlnet.nl_layer3.W.0.bias", 
            "nlnet.nl_layer3.residual_conv.weight", "nlnet.nl_layer3.residual_conv.bias", 
]

In [2]:
old_key = ["nlnet.nl_layer1.W.0.weight", "nlnet.nl_layer1.W.0.bias",
            "nlnet.nl_layer2.W.0.weight", "nlnet.nl_layer2.W.0.bias", 
            "nlnet.nl_layer3.W.0.weight", "nlnet.nl_layer3.W.0.bias"]
new_key = ["nlnet.nl_layer1.W.weight", "nlnet.nl_layer1.W.bias", "nlnet.nl_layer2.W.weight", "nlnet.nl_layer2.W.bias", "nlnet.nl_layer3.W.weight", "nlnet.nl_layer3.W.bias"]
for i in range(len(old_key)):
    new = new_key[i]
    old = old_key[i]
    dict[new] = dict.pop(old)

In [3]:

torch.save(dict, '../../video-polyp-seg/LSI-Net/snapshot/model_changed.pth')
#验证修改是否成功
# changed_dict = torch.load('../../video-polyp-seg/LSI-Net/snapshot/model_changed.pth')


# calculate the size of dataset

In [22]:
import os
cases = 0
frames = 0
path = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestEasyDataset/Seen/Frame/'
cases = len(list(os.listdir(path)))
for case in os.listdir(path):
    for file in os.listdir(path+case):
        img = os.path.join(path,case,file)
        if img.endswith('jpg'):
            frames += 1
        else:
            print(img)

print(cases,frames)

33 4719


In [19]:
import os
cases = 0
frames = 0
path = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestEasyDataset/Unseen/Frame/'
cases = len(list(os.listdir(path)))
for case in os.listdir(path):
    for file in os.listdir(path+case):
        img = os.path.join(path,case,file)
        if img.endswith('jpg'):
            frames += 1
        else:
            print(img)
print(cases,frames)

/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestEasyDataset/Unseen/Frame/case24_4/.DS_Store
/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestEasyDataset/Unseen/Frame/case24_4/._.DS_Store
86 12351


In [20]:
import os
cases = 0
frames = 0
path = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestHardDataset/Seen/Frame/'
cases = len(list(os.listdir(path)))
for case in os.listdir(path):
    for file in os.listdir(path+case):
        img = os.path.join(path,case,file)
        if img.endswith('jpg'):
            frames += 1
        else:
            print(img)
print(cases,frames)

17 3882


In [21]:
import os
cases = 0
frames = 0
path = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestHardDataset/Unseen/Frame/'
cases = len(list(os.listdir(path)))
for case in os.listdir(path):
    for file in os.listdir(path+case):
        img = os.path.join(path,case,file)
        if img.endswith('jpg'):
            frames += 1
        else:
            print(img)
print(cases,frames)

37 8640
